In [42]:
from selenium import webdriver
import time
import os
from gsheets import Sheets
import pandas as pd
import operator
import mysql.connector
import shutil
from shutil import copyfile
import os.path
import csv
from gspread_pandas import Spread, Client 
from oauth2client.service_account import ServiceAccountCredentials
import pprint
import gspread
import math
import random 


In [43]:
#reading credientials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

#wd for authentiction file
os.chdir('/Users/innovatus6')
creds = ServiceAccountCredentials.from_json_keyfile_name('linkupdater-631ef6e77556.json', scope)


In [44]:
#getting permission 
client = gspread.authorize(creds)

In [45]:
#opening production sheet
requests= client.open("Master Data Builds").worksheet("Priority Builds -requests")
primary= client.open("Master Data Builds").worksheet("Priority Builds")

In [59]:
df = pd.DataFrame(requests.get_all_records())

In [60]:
harvestfield=df.loc[operator.and_(df.type == 'notfounder', df.type != 'used')]

In [61]:
harvestfield=harvestfield.reset_index(drop=True)
harvestfield

,event,founderid,type,link
0,cf_fran,"titles, from founder 2",notfounder,https://www.linkedin.com/sales/search/people?c...
1,cf_fran,"titles, from founder 3",notfounder,https://www.linkedin.com/sales/search/people?c...
2,cf_fran,"titles, from founder 4",notfounder,https://www.linkedin.com/sales/search/people?c...
3,cf_fran,"titles, from founder 5",notfounder,https://www.linkedin.com/sales/search/people?c...
4,cf_fran,"titles, from founder 6",notfounder,https://www.linkedin.com/sales/search/people?c...
5,cf_fran,"titles, from founder 7",notfounder,https://www.linkedin.com/sales/search/people?c...
6,cf_fran,"titles, from founder 8",notfounder,https://www.linkedin.com/sales/search/people?c...
7,cf_fran,"titles, from founder 9",notfounder,https://www.linkedin.com/sales/search/people?c...
8,cf_fran,"titles, from founder 10",notfounder,https://www.linkedin.com/sales/search/people?c...
9,cf_fran,"titles, from founder 11",notfounder,https://www.linkedin.com/sales/search/people?c...


In [49]:
##the below code fills in the registration page and submits it

In [50]:
#toggleing linkedin login page
web = webdriver.Chrome('/Users/innovatus6/chromedriver')
time.sleep(random.uniform(4,7))
web.get("https://www.linkedin.com/home")

In [51]:
#pressing linkedin login button
button=web.find_element_by_xpath('/html/body/nav/div/a[2]')
time.sleep(random.uniform(4,7))
web.execute_script("arguments[0].click();", button)
time.sleep(random.uniform(14,17))

In [52]:
#email
email="noah@innovatusmedia.com.au"
emailslot=web.find_element_by_xpath('//*[@id="username"]')
time.sleep(random.uniform(1,3))
emailslot.send_keys(email)


#pw
pw="Innov8us"
firstslot=web.find_element_by_xpath('//*[@id="password"]')
time.sleep(random.uniform(1,3))
firstslot.send_keys(pw)


#submitting
buttonsub=web.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button')
time.sleep(random.uniform(4,7))
web.execute_script("arguments[0].click();", buttonsub)
time.sleep(random.uniform(4,15))

In [53]:
#skipping
buttonskip=web.find_element_by_xpath('//*[@id="ember15"]/button[2]')
time.sleep(random.uniform(6,7))
buttonskip.click()


In [54]:
#toggling sales nav page
time.sleep(random.uniform(4,7))
web.get("https://www.linkedin.com/uas/login?session_redirect=/sales&fromSignIn=true&trk=navigator")

In [55]:
#submitting pw again

pw="Innov8us"
firstslot=web.find_element_by_xpath('//*[@id="password"]')
time.sleep(random.uniform(1,7))
firstslot.send_keys(pw)
time.sleep(random.uniform(4,5))


In [56]:
#clicking sign in button
buttonsub=web.find_element_by_xpath('//*[@id="app__container"]/main/div[2]/form/div[3]/button')
time.sleep(random.uniform(4,7))
web.execute_script("arguments[0].click();", buttonsub)
time.sleep(random.uniform(4,7))

In [57]:
#parameters for the below loop

para=pd.DataFrame(data=([["function","92","0"],["industry","77","0"],["seniority","87","+1"]]), columns=['type','id','listmod'])

para

,type,id,listmod
0,function,92,0
1,industry,77,0
2,seniority,87,+1


In [ ]:
#editting code to accomdate all 3 types of filters
#j represents the jth unsplit link
for j in range(0,len(harvestfield.link)):

    #parsing link
    linktext=harvestfield.link[j]
    
    #toggling primivative link
    web.get(linktext)   
   
    link_count = web.find_element_by_xpath('//*[@id="search-spotlight-tab-ALL"]/span[1]')

    #if the initial link is already a integar then we can skip to the next iteration
    if link_count.text.isdigit():
        print("the"+str(j) + " link is done")
        #defining the row to append
        row=['',harvestfield.event[j],harvestfield.founderid[j],'','','',int(link_count.text),linktext]
        #excluding the recently used fn

        #appending to the primary sheet
        primary.append_row(row,table_range='A1')     
        
        #updating primary sheet
        requests.update_cell(int(df[df.iloc[:,3]==harvestfield.link[j]].index.to_numpy())+2,3,"used")
        
        #skipping the while loop because there is nothing in need of breaking down
        continue

    #for all 3 types of filters...
    for n in range(0,3):
        #extracts all the big-ish small links from the primitive link
        while not link_count.text.isdigit():

        #clicking type button 
            buttonsub=web.find_element_by_xpath('//*[@id="ember'+str(para.id[n])+'"]/div')
            time.sleep(random.uniform(4,7))
            web.execute_script("arguments[0].click();", buttonsub)
            time.sleep(random.uniform(14,17))


            #fnlist is a dataframe of members of the type of parameters 
            fnlist=pd.DataFrame(web.find_element_by_xpath('//*[@id="ember'+str(int(para.id[n])-1)+'"]/div[3]').text.split('\n'), columns = ["linksize"])


            for i in range(0,len(fnlist)):

                #defining the start and end string locations
                start=fnlist.iloc[i].str.find("(")[0]+1
                end=len(fnlist.iloc[i][0])-1


                #the first element wont have ( in it, so we deal with it here
                fnlist.iloc[0]=0

                #removing garbage strings
                try:
                    fnlist.iloc[i]=fnlist.iloc[i].str.slice(start,end)

                except:
                    fnlist.iloc[i]=0;

                #trying to convert elements into integars
                try:
                    fnlist.iloc[i]=int(fnlist.iloc[i])

                except:
                    fnlist.iloc[i]=0;





            #this below line of code allows us to use idxmax()        
            fnlist['linksize'] = pd.to_numeric(fnlist['linksize'])

            #this is the biggest boi out of all the options under the current catergory
            idmax=fnlist["linksize"].idxmax()+int(para.listmod[n])



            valuemax=fnlist["linksize"].max()

            if valuemax < 100:
                break




            #we can select from this list by index, starting at i=1
            fncount=web.find_element_by_xpath('//*[@id="ember'+str(int(para.id[n])-1)+'"]/div[3]/ol/li['+str(idmax)+']/button')


            #locating the button corresponding to the maxid on web
            buttonsub=web.find_element_by_xpath('//*[@id="ember'+str(para.id[n])+'"]/div')
            time.sleep(random.uniform(4,7))


            #then clicking the button 
            web.execute_script("arguments[0].click();", fncount)
            time.sleep(random.uniform(3,9))


            #storing and redfining the url at work
            uploadurl=web.current_url
            #defining the row to append
            row=['',harvestfield.event[j],harvestfield.founderid[j],para.type[n],'','',int(valuemax),uploadurl]
            #appending to the primary sheet
            primary.append_row(row,table_range='A1')



            #updating url with exclusion  


            if "&"+para.type[n]+"Excluded=" in uploadurl:
                url=uploadurl.replace("&"+para.type[n]+"Included=","%2C")

            else:
                url=uploadurl.replace(para.type[n]+"Included", para.type[n]+"Excluded")

            #toggling updated link    
            web.get(url)

            #getting initial link count
            link_count = web.find_element_by_xpath('//*[@id="search-spotlight-tab-ALL"]/span[1]')





    #if the full exclusion link count now is digital, we can paste this final row and toggle the next link         
    print("the"+str(j) + " link is done")
    #defining the row to append
    row=['',harvestfield.event[j],harvestfield.founderid[j],para.type[n],'','',int(link_count.text),url]
    #excluding the recently used fn

    #appending to the primary sheet    
    primary.append_row(row,table_range='A1')

    #updating the requests sheet when a source link is done
    requests.update_cell(int(df[df.iloc[:,3]==harvestfield.link[j]].index.to_numpy())+2,3,"used")

the0 link is done
the1 link is done
the2 link is done
the3 link is done
the4 link is done
the5 link is done
the6 link is done
the7 link is done
the8 link is done
